# Question from Reddit
https://www.reddit.com/r/learnpython/comments/qmngli/reddit_data/?utm_source=share&utm_medium=web2x&context=3

In [1]:
from secret_services import reddit

subreddit_name="learnpython"
subreddit=reddit.subreddit(subreddit_name)

#Can change window size(s) to compare top links over tiem
for post in subreddit.top("week",limit=10):
    post

In [4]:
def get_post_text(post):
    if post.selftext:
        return post.selftext
    else:
        return post.url
from praw.models import MoreComments

#Improved version that fixes the MoreComments bug
def traverse_post(post):
    comments=[]
    for comment in post.comments:
        comments+=recursive_replies(comment, level=1)
    return comments

def recursive_replies(reply,level):
    #Also return level in case we want to stop after level 3 comments and for ease of the printing 
    comments=[]
    #Funky MoreComments code checked manually with permalinks and seems right
    if isinstance(reply, MoreComments):#https://praw.readthedocs.io/en/stable/code_overview/models/more.html
        replies=reply.comments()
        level-=1
    else:
        replies=reply.replies
        comments+=[(reply, level)]
    for r in replies:
        comments+=recursive_replies(r,level+1)
    return comments

In [15]:
import pandas as pd

word_to_check="use"
post_with_comments=[]
for post in subreddit.top("week",limit=10):
    for comment,level in traverse_post(post):
        if word_to_check in comment.body:
            post_with_comments.append(
                {"comment_id": comment.id, "comment_text": comment.body,"score": comment.score,"post": post.id,
                 "comment_nesting_level": level
                }
            )
df=pd.DataFrame(post_with_comments)
df

,comment_id,comment_text,score,post,comment_nesting_level
0,hiiklul,this book is so damned good. the data visualiz...,21,qibvec,2
1,hijkssv,\ri want to learn python and programming langu...,4,qibvec,2
2,him9dnk,Is that a yellow book? Pale yellow? I think t...,1,qibvec,2
3,hiipccg,I always recommend trying to do some projects....,23,qibvec,1
4,hijifum,I always recommend people code solitaire after...,10,qibvec,2
...,...,...,...,...,...
171,himmicp,Have you upgraded python? If you're using a ne...,1,qiymbc,1
172,hina1wp,I really like Ninja IDE for Linux. Super ligh...,1,qiymbc,1
173,hinsul7,If you're a beginner still learning functions ...,1,qiymbc,1
174,hintz63,"If programming directly on the rpi, keep using...",1,qiymbc,1


In [17]:
len(df[df['comment_text'].str.contains(word_to_check)])==len(df)#Success!

True